# Instalação das bibliotecas


In [8]:
# Importando bibliotecas necessárias
import numpy as np
import pandas as pd
import tensorflow as tf
import gdown
import plotly.express as px
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.optimizers import Adam

# Carregamento dos dados

O trecho abaixo automatiza o processo de download e leitura de arquivos CSV armazenados no Google Drive.

In [9]:
arquivo_destino_base = "dataset_{}.csv"

ids = {
    "credit_card": "1UgWQUgSzTTdV4h9IHVrrLpw7cp5flz1n",
}

dataframes = {}

for key, file_id in ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_destino_base.format(key)

    gdown.download(url, arquivo_destino, quiet=False)
    df = pd.read_csv(arquivo_destino, sep=";")
    dataframes[key] = df

Downloading...
From (original): https://drive.google.com/uc?id=1UgWQUgSzTTdV4h9IHVrrLpw7cp5flz1n
From (redirected): https://drive.google.com/uc?id=1UgWQUgSzTTdV4h9IHVrrLpw7cp5flz1n&confirm=t&uuid=1d1c0bfb-4ee0-486a-b4b4-38566e10615d
To: /content/dataset_credit_card.csv
100%|██████████| 151M/151M [00:03<00:00, 42.3MB/s]


In [10]:
df = pd.read_csv("/content/dataset_credit_card.csv", delimiter=",")

In [11]:
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


# Organizar o dataset

Neste trecho de código, realizamos a preparação dos dados para a construção de um modelo de aprendizado de máquina.

In [12]:
# Separando características (X) e rótulo (y)
X = df.drop(columns=['Class'])  # Características
y = df['Class']  # Rótulo

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalizando os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Modelo 1

Este trecho de código implementa o primeiro modelo de rede neural para a tarefa de classificação binária. Utilizei o Keras com a API Sequential para construir o modelo.

In [13]:
# Modelo 1: Seu modelo original
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=64,
                    validation_data=(X_test, y_test))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 25s 6ms/step - accuracy: 0.9191 - loss: 0.1921 - val_accuracy: 0.9993 - val_loss: 0.0097
Epoch 2/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9992 - loss: 0.0055 - val_accuracy: 0.9993 - val_loss: 0.0104
Epoch 3/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9992 - loss: 0.0042 - val_accuracy: 0.9993 - val_loss: 0.0101
Epoch 4/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9992 - loss: 0.0040 - val_accuracy: 0.9993 - val_loss: 0.0100
Epoch 5/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - accuracy: 0.9992 - loss: 0.0040 - val_accuracy: 0.9993 - val_loss: 0.0082
Epoch 6/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.9992 - loss: 0.0039 - val_accuracy: 0.9993 - val_loss: 0.0082
Epoch 7/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9993 - loss: 0.0039 - val_accuracy: 0.9993 - val_loss: 0.0073
Epoch 8/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9993 - loss: 0.

# Modelo 2

Este trecho de código apresenta uma segunda abordagem de modelo de rede neural, onde introduzimos uma arquitetura diferente com uma maior quantidade de neurônios e camadas de normalização em lotes adicionais.

In [14]:
# Modelo 2: Modelo com Batch Normalization adicional e arquitetura diferente
model_batchnorm = Sequential([
    Dense(128, input_dim=X_train.shape[1]),
    BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    Dropout(0.5),  # Adicionando Dropout para evitar overfitting
    Dense(64),
    BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Como é um problema binário, usamos sigmoid
])

# Compilar o modelo com Batch Normalization
model_batchnorm.compile(optimizer='adam',
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

# Treinar o modelo
history_batchnorm = model_batchnorm.fit(X_train, y_train,
                                        epochs=50,
                                        batch_size=64,
                                        validation_data=(X_test, y_test))

Epoch 1/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9901 - loss: 0.0461 - val_accuracy: 0.9993 - val_loss: 0.0045
Epoch 2/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9993 - loss: 0.0046 - val_accuracy: 0.9993 - val_loss: 0.0045
Epoch 3/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9994 - loss: 0.0037 - val_accuracy: 0.9993 - val_loss: 0.0050
Epoch 4/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9994 - loss: 0.0039 - val_accuracy: 0.9993 - val_loss: 0.0041
Epoch 5/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9994 - loss: 0.0033 - val_accuracy: 0.9993 - val_loss: 0.0044
Epoch 6/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9993 - loss: 0.0033 - val_accuracy: 0.9993 - val_loss: 0.0041
Epoch 7/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9994 - loss: 0.0032 - val_accuracy: 0.9993 - val_loss: 0.0044
Epoch 8/50
3561/3561 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9993 - loss: 0.

# Comparação entre os modelos

Neste trecho de código, avaliamos o desempenho dos dois modelos treinados no conjunto de teste, comparando suas acurácias.


In [15]:
# Acurácia no conjunto de teste para ambos os modelos
test_loss1, test_accuracy1 = model.evaluate(X_test, y_test)
test_loss2, test_accuracy2 = model_batchnorm.evaluate(X_test, y_test)

print(f"Acurácia do Modelo 1: {test_accuracy1}")
print(f"Acurácia do Modelo 2: {test_accuracy2}")

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9994 - loss: 0.0043
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9995 - loss: 0.0021
Acurácia do Modelo 1: 0.9993504285812378
Acurácia do Modelo 2: 0.9994031190872192


# Gráfico com a comparação

Neste trecho de código, preparamos os dados de acurácia dos modelos durante o treinamento e validação para visualização usando o Plotly Express.

In [16]:
# Preparando os dados para Plotly Express
history_df = pd.DataFrame({
    'epoch': range(1, len(history.history['accuracy']) + 1),
    'Acurácia de Treinamento - Modelo 1': history.history['accuracy'],
    'Acurácia de Validação - Modelo 1': history.history['val_accuracy'],
    'Acurácia de Treinamento - Modelo 2': history_batchnorm.history['accuracy'],
    'Acurácia de Validação - Modelo 2': history_batchnorm.history['val_accuracy']
})

# Transformando para o formato longo para Plotly Express
history_df = history_df.melt(id_vars='epoch', var_name='Tipo', value_name='Acurácia')

In [17]:
# Plotando a acurácia com Plotly Express
fig = px.line(history_df, x='epoch', y='Acurácia', color='Tipo',
              labels={'epoch': 'Épocas', 'Acurácia': 'Acurácia'},
              title='Comparação da Acurácia durante o Treinamento e Validação')

# Exibindo o gráfico
fig.show()

# AUC-ROC

Neste trecho de código, realizamos a etapa de previsão e avaliação da performance dos modelos utilizando a métrica AUC-ROC, que é amplamente utilizada para medir o desempenho de classificadores binários.


In [18]:
# Previsões no conjunto de teste
y_pred1 = (model.predict(X_test) > 0.5).astype("int32")
y_pred2 = (model_batchnorm.predict(X_test) > 0.5).astype("int32")

# Calculando AUC-ROC para ambos os modelos
auc_roc1 = roc_auc_score(y_test, y_pred1)
auc_roc2 = roc_auc_score(y_test, y_pred2)

print(f"AUC-ROC do Modelo 1: {auc_roc1}")
print(f"AUC-ROC do Modelo 2: {auc_roc2}")

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step
AUC-ROC do Modelo 1: 0.9079962000275631
AUC-ROC do Modelo 2: 0.9131158266626853


# Relatório

Neste trecho de código, geramos e exibimos relatórios de classificação para ambos os modelos treinados, utilizando a função classification_report do scikit-learn.

In [19]:
# Relatório de classificação para ambos os modelos
print("Modelo 1 - Relatório de Classificação")
print(classification_report(y_test, y_pred1))

print("Modelo 2 - Relatório de Classificação")
print(classification_report(y_test, y_pred2))

Modelo 1 - Relatório de Classificação
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.81      0.82      0.81        98

    accuracy                           1.00     56962
   macro avg       0.90      0.91      0.91     56962
weighted avg       1.00      1.00      1.00     56962

Modelo 2 - Relatório de Classificação
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.83      0.83      0.83        98

    accuracy                           1.00     56962
   macro avg       0.91      0.91      0.91     56962
weighted avg       1.00      1.00      1.00     56962



# Resultados obtidos com análise


### Acurácia de Treinamento e Validação
Conforme mostrado no gráfico, a acurácia de treinamento e validação para ambos os modelos é muito próxima de 1 (100%), o que sugere que os modelos estão muito bem ajustados aos dados. No entanto, acurácias tão altas podem indicar que o modelo está se ajustando muito bem ao conjunto de dados, o que levanta a possibilidade de overfitting, embora a pequena diferença entre acurácia de treinamento e validação sugira que isso não é um grande problema.

### AUC-ROC
O AUC-ROC do **Modelo 1** foi de 0.908, enquanto o AUC-ROC do **Modelo 2** foi ligeiramente superior, com um valor de 0.913. Ambos os valores indicam um excelente desempenho na discriminação entre as classes. O AUC-ROC próximo de 1 indica que os modelos são muito bons em distinguir entre a classe positiva e a classe negativa. A ligeira melhoria no AUC-ROC do segundo modelo sugere que a adição de camadas de Batch Normalization e a arquitetura modificada proporcionaram uma pequena vantagem em termos de capacidade de discriminação.

### Relatório de Classificação
Os relatórios de classificação fornecem uma visão mais detalhada sobre o desempenho de cada modelo em termos de `precision`, `recall`, e `f1-score`:

- **Modelo 1**:
  - **Precisão**: O modelo alcançou precisão perfeita na classe 0 (não fraude) e 81% na classe 1 (fraude).
  - **Recall**: O recall para a classe 1 foi de 82%, indicando que o modelo conseguiu identificar 82% das fraudes corretamente.
  - **F1-Score**: A pontuação F1, que é a média harmônica entre precisão e recall, foi de 81% para a classe 1.

- **Modelo 2**:
  - **Precisão**: O segundo modelo teve uma ligeira melhoria, alcançando 83% de precisão na classe 1.
  - **Recall**: O recall também melhorou para 83%, mostrando que o modelo identificou corretamente uma fração maior de fraudes.
  - **F1-Score**: O F1-score subiu para 83% na classe 1, indicando uma melhoria geral na capacidade de prever fraudes.

### Conclusão
Os resultados indicam que ambos os modelos têm um desempenho excelente em termos de classificação de transações fraudulentas, com uma leve vantagem para o segundo modelo, que foi capaz de melhorar tanto a precisão quanto o recall para a classe de fraude (classe 1). Esta melhoria, embora pequena, pode ser significativa dependendo do contexto de aplicação, onde a identificação precisa de fraudes é crucial. O uso adicional de Batch Normalization e uma arquitetura ligeiramente mais complexa no Modelo 2 parece ter proporcionado essa vantagem.